In [2]:
from ipywidgets import IntSlider, Layout
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rioxarray
import leafmap.leafmap as leafmap
import geopandas as gpd
from ipyleaflet import Map, basemaps, ImageOverlay, projections, GeoJSON, GeoData, WMSLayer, Marker
import odc.geo.xr
# from sidecar import Sidecar
# import xarray_leaflet
# import odc.ui
# import hvplot
# import os

In [ ]:
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f'{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}'

## IPYLEAFLET

In [3]:
# Create point file from geojson for display on leaflet map

ship_track = gpd.read_file('/g/data/jk72/sc0554/miz/data/miz_stations.geojson')

geo_data = GeoData(geo_dataframe = ship_track,
    style={'color': 'black', 'radius':6, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'Release')

In [4]:
# Create custom projection with bounds for Polar view WMS layer

POLAR3031 = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds =[[-2822131.5,-3057369.25],[3744213.75,3822194.25]]
)

In [5]:
# Function for plotting xarray dataarray on leaflet maps
def polar_map(da, epsg):
    da = da.odc.reproject(epsg)
    vmin, vmax = np.nanpercentile(da.data, [2, 98])
    rgba = da.odc.colorize(vmin=vmin, vmax=vmax)
    data_url = rgba.odc.compress(as_data_url=True)
    (x1, y1), _, (x2, y2) = rgba.odc.geobox.extent.exterior.to_crs("epsg:4326").points[:3]
    bounds = [[y1, x1], [y2, x2]]
    return data_url, bounds

In [6]:
# Plotting raster data
yy = xr.open_dataset("/g/data/jk72/MIZ/Bremen/netcdf/asi-AMSR2-s6250-20190902-v5.4.nc")

yy=yy['z']

# data was plotting upside down so inverting
yy= yy[::-1]

## Assign CRS
yy = odc.geo.xr.assign_crs(yy, "epsg:3976")

# Create data url from custom function
data_url, bounds = polar_map(yy, epsg="EPSG:3031")

# Create image overlay using bounds and image data
raster = ImageOverlay(url=data_url, bounds=bounds,opacity=0.7)

graticule = WMSLayer(
    url='http://geos.polarview.aq/geoserver/wms',
    layers='polarview:graticuleS',
    format='image/png',
    transparent=True,
    attribution='Polarview',
    crs=POLAR3031
)

coastline = WMSLayer(
    url='http://geos.polarview.aq/geoserver/wms',
    layers='polarview:coastS10',
    format='image/png',
    transparent=True,
    attribution='Polarview',
    crs=POLAR3031
)

In [7]:
# Create map layer
spsLayout=Layout(width='800px', height='800px')
m = Map(basemap=basemaps.NASAGIBS.BlueMarble3031, center=(-90, 0), zoom=1, crs=projections.EPSG3031, layout=spsLayout)

# Add layers
m.add_layer(raster)
m.add_layer(geo_data)
# m.add_layer(coastline)
m.add_layer(graticule)

# Display map
m

Map(center=[-90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [ ]:
test = xr.open_dataset("/g/data/jk72/MIZ/NSIDC/G02202_V4/seaice_conc_monthly_sh_197811_202112_v04r00.nc")
test = odc.geo.xr.assign_crs(test, "epsg:3976")
# test = test['cdr_seaice_conc_monthly'].odc.reproject("epsg:3031")

display(test.odc.geobox)

In [ ]:
test = xr.open_dataset("/g/data/jk72/MIZ/Bremen/netcdf/asi-AMSR2-s6250-20190202-v5.4.nc")
test = odc.geo.xr.assign_crs(test, "epsg:3976")
test = test['z'].odc.reproject("epsg:3031")
display(test.odc.geobox)

In [ ]:
# Plot monthly netcdf data

spsLayout=Layout(width='800px', height='800px')

yy = xr.open_dataset("/g/data/jk72/MIZ/NSIDC/G02202_V4/seaice_conc_monthly_sh_197811_202112_v04r00.nc")
yy=yy['cdr_seaice_conc_monthly'].isel(tdim=500).drop('time')
yy = odc.geo.xr.assign_crs(yy, "epsg:3976")

data_url, bounds = polar_map(yy, epsg="EPSG:3031")

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3031, center=(-90, 0), zoom=1, crs=projections.EPSG3031, layout=spsLayout)

m.add_layer(ImageOverlay(url=data_url, bounds=bounds,opacity=0.7))

m

In [ ]:
# NSIDC sea ice concentration data

import numpy as np
from ipyleaflet import Map, basemaps, ImageOverlay, projections
import rioxarray as rio
from ipywidgets import Layout
import xarray as xr
import odc.geo.xr

def polar_map(da, epsg):
    da = da.odc.reproject(epsg)
    vmin, vmax = np.nanpercentile(da.data, [2, 98])
    rgba = da.odc.colorize(vmin=vmin, vmax=vmax)
    data_url = rgba.odc.compress(as_data_url=True)
    (x1, y1), _, (x2, y2) = rgba.odc.geobox.extent.exterior.to_crs("epsg:4326").points[:3]
    bounds = [[y1, x1], [y2, x2]]
    return data_url, bounds

spsLayout=Layout(width='800px', height='800px')

path = '/g/data/jk72/sc0554/miz/data/S_201905_concentration_v3.0.tif'

yy = rio.open_rasterio(path, mask=True).isel(band=0)

yy = xr.where(yy > 2500, np.nan, yy, keep_attrs=True)

data_url, bounds = polar_map(yy, epsg="EPSG:3031")

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3031, center=(-90, 0), zoom=1, crs=projections.EPSG3031, layout=spsLayout)

m.add_layer(ImageOverlay(url=data_url, bounds=bounds, opacity=0.7))

m

In [ ]:
# Sea ice edge example

from ipyleaflet import Map, WMSLayer, basemaps

spsLayout=Layout(width='800px', height='800px')

POLAR3031 = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds =[[-2822131.5,-3057369.25],[3744213.75,3822194.25]]
)

wms = WMSLayer(
    url='http://geos.polarview.aq/geoserver/wms',
    layers='polarview:iceedgeS15',
    format='image/png',
    transparent=True,
    attribution='Polarview',
    crs=POLAR3031
)

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3031, center=(-90, 0), zoom=1, crs=projections.EPSG3031, layout=spsLayout)

m.add_layer(wms)

m

## Leafmap

In [ ]:
data = xr.open_dataset('/g/data/jk72/MIZ/processed/DEA GeoMAD (Landsat 8 OLI-TIRS) clip.tiff', engine='rasterio').isel(band=1)

In [ ]:
data.geo

In [ ]:
# South Polar sterographic
spsLayout=Layout(width='800px', height='800px')

m = leafmap.Map()

data = '/g/data/jk72/MIZ/processed/DEA GeoMAD (Landsat 8 OLI-TIRS) clip.tiff'
m.add_geotiff(data, bands=[1], layer_name='conc')

# url = 'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/s6250/2015/jan/Antarctic/asi-AMSR2-s6250-20150101-v5.4.tif'
# leafmap.cog_bounds(url)
# leafmap.cog_center(url)
# leafmap.cog_bands(url)
# leafmap.cog_tile(url)
# m.add_cog_layer(url, name="sea ice conc")
# 

m

In [ ]:
# South Polar sterographic
spsLayout=Layout(width='800px', height='800px')

m = leafmap.Map(center=(-90, 0),
        zoom=0,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

data = '/g/data/jk72/MIZ/Bremen/geotiff/asi-AMSR2-s6250-20180505-v5.4.tif'



m.add_geotiff(data, bands=[1], layer_name='conc')

# url = 'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/s6250/2015/jan/Antarctic/asi-AMSR2-s6250-20150101-v5.4.tif'
# leafmap.cog_bounds(url)
# leafmap.cog_center(url)
# leafmap.cog_bands(url)
# leafmap.cog_tile(url)
# m.add_cog_layer(url, name="sea ice conc")
# 

# filename = 'DEA GeoMAD (Landsat 8 OLI-TIRS) clip.tiff'
# filename = 'asi-AMSR2-s6250-20150101-v5.4.tif'
# filename = 'S_201905_concentration_v3.0.tif'
# m.add_raster(filename, bands=[1], layer_name='conc')
# m.add_geotiff(filename, layer_name='conc')
# m.add_netcdf('DEA GeoMAD (Landsat 8 OLI-TIRS) clip.nc')
# m.add_local_tile(filename)

# url = 'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/s6250/2015/jan/Antarctic/asi-AMSR2-s6250-20150101-v5.4.tif'
# url = 'ttps://masie_web.apps.nsidc.org/pub//DATASETS/NOAA/G02135/'
# leafmap.cog_bounds(url)
# leafmap.cog_center(url)
# leafmap.cog_bands(url)
# leafmap.cog_tile(url)
# m.add_cog_layer(url, name="sea ice conc")
# m.add_remote_tile(url)
# m.add_tile_layer(url, name = 'a', attribution='y')

m

In [1]:
# LEAFMAP WMS

# South Polar sterographic
spsLayout=Layout(width='900px', height='1200px')

POLAR3031 = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds =[[-2822131.5,-3057369.25],[3744213.75,3822194.25]]
)

m = leafmap.Map(
    center=(-90, 0),
        zoom=1,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

m.add_wms_layer(url='http://geos.polarview.aq/geoserver/wms', layers='polarview:iceedgeS15', format='image/png',  transparent=True, attribution='Polarview', crs=POLAR3031)

m

NameError: name 'Layout' is not defined

In [ ]:
# edited image overlay function

def image_overlay(self, url, bounds, name):
    """Overlays an image from the Internet or locally on the map.
    Args:
        url (str): http URL or local file path to the image.
        bounds (tuple): bounding box of the image in the format of (lower_left(lat, lon), upper_right(lat, lon)), such as ((13, -130), (32, -100)).
        name (str): name of the layer to show on the layer control.
    """
    from base64 import b64encode
    from PIL import Image, ImageSequence
    from io import BytesIO

    try:
        if not url.startswith("http"):

            # if not os.path.exists(url):
            #     print("The provided file does not exist.")
            #     return

            ext = os.path.splitext(url)[1][1:]  # file extension
            image = Image.open(url)

            f = BytesIO()
            if ext.lower() == "gif":
                frames = []
                # Loop over each frame in the animated image
                for frame in ImageSequence.Iterator(image):
                    frame = frame.convert("RGBA")
                    b = BytesIO()
                    frame.save(b, format="gif")
                    frame = Image.open(b)
                    frames.append(frame)
                frames[0].save(
                    f,
                    format="GIF",
                    save_all=True,
                    append_images=frames[1:],
                    loop=0,
                )
            else:
                image.save(f, ext)

            data = b64encode(f.getvalue())
            data = data.decode("ascii")
            url = "data:image/{};base64,".format(ext) + data
        img = ipyleaflet.ImageOverlay(url=url, bounds=bounds, name=name)
        self.add_layer(img)
    except Exception as e:
        raise Exception(e)

In [ ]:
# Leafmap and raster data

def polar_map(da, epsg):
    da = da.odc.reproject(epsg)
    vmin, vmax = np.nanpercentile(da.data, [2, 98])
    rgba = da.odc.colorize(vmin=vmin, vmax=vmax)
    data_url = rgba.odc.compress(data_url=True)
    (x1, y1), _, (x2, y2) = rgba.odc.geobox.extent.exterior.to_crs("epsg:4326").points[:3]
    bounds = [[y1, x1], [y2, x2]]
    return data_url, bounds, rgba

spsLayout=Layout(width='800px', height='800px')

path = '/g/data/jk72/sc0554/miz/data/S_201905_concentration_v3.0.tif'

yy = rio.open_rasterio(path, mask=True).isel(band=0)

yy = xr.where(yy > 2500, np.nan, yy, keep_attrs=True)

data_url, bounds, rgba = polar_map(yy, epsg="EPSG:3031")

m = leafmap.Map(center=(-90, 0),
        zoom=1,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

m.add_layer(ImageOverlay(url=data_url, bounds=bounds, name='layer'))

m

In [ ]:
m.image_overlay(url=data_url, bounds=bounds, name='layer')

## Xarray leaflet

In [ ]:
from xarray_leaflet.transform import passthrough
from ipyleaflet import Map
import xarray_leaflet

In [ ]:
da = rioxarray.open_rasterio('S_201905_concentration_v3.0.tif')
da = da.isel(band=0).drop('band'

In [ ]:
m = Map()
l = da.leaflet.plot(m, transform1=passthrough)
m

## HVPLOT

In [ ]:
import hvplot.xarray

In [ ]:
data = xr.open_dataset('/g/data/jk72/MIZ/processed/asi2_monthly.nc')

In [ ]:
data=data.isel(time=slice(0,3))

In [ ]:
data['conc'].hvplot(groupby='time',frame_height=500, frame_width=500)